<a href="https://colab.research.google.com/github/Pheonix64/Android-SqliteDBProject/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
!wget https://data.broadinstitute.org/bbbc/BBBC041/malaria.zip

--2024-10-18 15:05:05--  http://vivax/
Resolving vivax (vivax)... failed: Name or service not known.
wget: unable to resolve host address ‘vivax’
--2024-10-18 15:05:05--  https://data.broadinstitute.org/bbbc/BBBC041/malaria.zip
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.68.137
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.68.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2259224287 (2.1G) [application/zip]
Saving to: ‘malaria.zip’

malaria.zip         100%[===================>]   2.10G  45.4MB/s    in 49s     

2024-10-18 15:05:55 (44.4 MB/s) - ‘malaria.zip’ saved [2259224287/2259224287]

FINISHED --2024-10-18 15:05:55--
Total wall clock time: 49s
Downloaded: 1 files, 2.1G in 49s (44.4 MB/s)


In [ ]:
!unzip malaria.zip

In [21]:
!wget https://github.com/andrealoddo/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis/archive/master.zip

--2024-10-18 15:27:58--  https://github.com/andrealoddo/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis/archive/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andrealoddo/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis/zip/refs/heads/master [following]
--2024-10-18 15:27:58--  https://codeload.github.com/andrealoddo/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ] 703.95M  23.5MB/s    in 30s     

2024

In [ ]:
!unzip master.zip

In [19]:
import torch
# Global veriables

class GlobalConfig:
        root_dir = '../content/malaria'
        train_json_path = '../content/malaria/training.json'
        test_json_path = '../content/malaria/test.json'
        seed = 1234
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def seed_everything(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)

config = GlobalConfig()
seed_everything(config.seed)

In [20]:
train_df = pd.read_json(config.train_json_path)
test_df = pd.read_json(config.test_json_path)
print("Train data Shape: ",train_df.shape,"\nTest data shape: ", test_df.shape)

Train data Shape:  (1208, 2) 
Test data shape:  (120, 2)


In [47]:
df = pd.concat([train_df, test_df]).reset_index(drop=True)
df.head()

,image,objects
0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':..."
1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ..."
2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ..."
3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ..."
4,{'checksum': 'ccef403e971460b86444cca669e68ca1...,"[{'bounding_box': {'minimum': {'r': 618, 'c': ..."


In [48]:
print(df['image'].sample().values)

[{'checksum': '74decede36a30e7c3d1005bae60adbf7', 'pathname': '/images/0ca25c88-457f-4f03-bbc1-98fb6663f1d1.png', 'shape': {'r': 1200, 'c': 1600, 'channels': 3}}]


In [49]:
df["checksum"] = df['image'].apply(lambda x: x["checksum"])
df["pathname"] = df['image'].apply(lambda x: x["pathname"][1:])
df["shape"] = df['image'].apply(lambda x: x["shape"])
df.head()

,image,objects,checksum,pathname,shape
0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':...",676bb8e86fc2dbf05dd97d51a64ac0af,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ...",1225a18efce159eddf7b0e80e0ea642c,images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ...",3eaf840523c30fdf38897ffa01e194eb,images/6b14c855-8561-417c-97a4-63fa552842fd.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ...",8a111dffacfa433029492780b9535091,images/13099edb-35d9-438f-b093-2cf2ebf9d255.png,"{'r': 1200, 'c': 1600, 'channels': 3}"
4,{'checksum': 'ccef403e971460b86444cca669e68ca1...,"[{'bounding_box': {'minimum': {'r': 618, 'c': ...",ccef403e971460b86444cca669e68ca1,images/2559636b-f01a-4414-93da-210c3b12d153.png,"{'r': 1200, 'c': 1600, 'channels': 3}"


In [50]:
df = pd.DataFrame(
    [
     dict(pathname=row['pathname'],
          shape=row['shape'],
          checksum=row['checksum'],
          **bb_info)
     for _, row in df.iterrows()
     for bb_info in row['objects']
     ]
     )
df

,pathname,shape,checksum,bounding_box,category
0,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,"{'minimum': {'r': 1057, 'c': 1440}, 'maximum':...",red blood cell
1,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,"{'minimum': {'r': 868, 'c': 1303}, 'maximum': ...",red blood cell
2,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,"{'minimum': {'r': 578, 'c': 900}, 'maximum': {...",red blood cell
3,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,"{'minimum': {'r': 304, 'c': 611}, 'maximum': {...",red blood cell
4,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,"{'minimum': {'r': 198, 'c': 881}, 'maximum': {...",red blood cell
...,...,...,...,...,...
86030,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,"{'minimum': {'r': 1047, 'c': 694}, 'maximum': ...",red blood cell
86031,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,"{'minimum': {'r': 543, 'c': 1381}, 'maximum': ...",red blood cell
86032,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,"{'minimum': {'r': 1041, 'c': 1725}, 'maximum':...",ring
86033,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,"{'minimum': {'r': 526, 'c': 374}, 'maximum': {...",ring


In [51]:
def extract_nested_list(it):
    """
    Recursively extracts all elements from a nested list or dictionary structure.

    This function traverses through nested lists and dictionaries, yielding
    individual elements or values. It can handle arbitrarily nested structures
    of lists and dictionaries.

    Parameters:
    -----------
    it : iterable (list or dict)
        The input iterable, which can be a list, dictionary, or a nested combination
        of both, from which elements will be extracted.

    Yields:
    -------
    Elements from the nested structure (int, float, str, etc.)
        Individual elements found within the nested lists or dictionaries.

    Example:
    --------
    Given the structure:
    nested_structure = [[1, 2], {'a': [3, 4], 'b': 5}, 6]

    The function will yield:
    1, 2, 3, 4, 5, 6
    """

    # Check if the current element is a list
    if isinstance(it, list):
        # Recursively yield elements from the sub-lists
        for sub_it in it:
            yield from extract_nested_list(sub_it)

    # Check if the current element is a dictionary
    elif isinstance(it, dict):
        # Recursively yield values from the dictionary
        for value in it.values():
            yield from extract_nested_list(value)

    # If it's not a list or dict, yield the element itself (base case)
    else:
        yield it

In [52]:
df['xmin'] = -1
df['ymin'] = -1
df['xmax'] = -1
df['ymax'] = -1

df[['ymin', 'xmin', 'ymax', 'xmax']] = np.stack(df['bounding_box'].apply(lambda x: np.array(list(extract_nested_list(x)))))
df.drop(columns=['bounding_box'], inplace=True)


In [53]:
df

,pathname,shape,checksum,category,xmin,ymin,xmax,ymax
0,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,red blood cell,1440,1057,1540,1158
1,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,red blood cell,1303,868,1403,971
2,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,red blood cell,900,578,1008,689
3,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,red blood cell,611,304,713,408
4,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,"{'r': 1200, 'c': 1600, 'channels': 3}",676bb8e86fc2dbf05dd97d51a64ac0af,red blood cell,881,198,1003,312
...,...,...,...,...,...,...,...,...
86030,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,red blood cell,694,1047,787,1145
86031,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,red blood cell,1381,543,1464,617
86032,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,ring,1725,1041,1880,1196
86033,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",7ec6cc1abc1c5a916150c34a673ba864,ring,374,526,508,653


In [54]:
df.drop(columns=['checksum','shape'], inplace=True)
df.rename(columns={'category':'parasite_type', 'pathname':'filename'}, inplace=True)
df

,filename,parasite_type,xmin,ymin,xmax,ymax
0,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1440,1057,1540,1158
1,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1303,868,1403,971
2,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,900,578,1008,689
3,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,611,304,713,408
4,images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,881,198,1003,312
...,...,...,...,...,...,...
86030,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,694,1047,787,1145
86031,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,1381,543,1464,617
86032,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,1725,1041,1880,1196
86033,images/887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,374,526,508,653


In [55]:
df['filename'] = df['filename'].apply(lambda x: x.split('/')[-1])
df

,filename,parasite_type,xmin,ymin,xmax,ymax
0,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1440,1057,1540,1158
1,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1303,868,1403,971
2,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,900,578,1008,689
3,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,611,304,713,408
4,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,881,198,1003,312
...,...,...,...,...,...,...
86030,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,694,1047,787,1145
86031,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,1381,543,1464,617
86032,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,1725,1041,1880,1196
86033,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,374,526,508,653


In [56]:
# Reorder the columns in the dataframe
new_column_order = ['filename', 'parasite_type', 'xmin', 'xmax', 'ymin', 'ymax']
df = df[new_column_order]

In [57]:
df

,filename,parasite_type,xmin,xmax,ymin,ymax
0,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1440,1540,1057,1158
1,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1303,1403,868,971
2,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,900,1008,578,689
3,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,611,713,304,408
4,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,881,1003,198,312
...,...,...,...,...,...,...
86030,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,694,787,1047,1145
86031,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,red blood cell,1381,1464,543,617
86032,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,1725,1880,1041,1196
86033,887cc81a-bae3-4360-a115-23ae05ac3a4f.jpg,ring,374,508,526,653


In [58]:
df_falc = pd.read_csv('/content/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis-master/Falciparum/mp-idb-falciparum.csv')
df_falc.head()

,filename,parasite_type,xmin,xmax,ymin,ymax
0,1305121398-0001-R_S.jpg,ring,919,887,76,67
1,1305121398-0001-R_S.jpg,schi,1246,1498,106,113
2,1305121398-0001-R_S.jpg,ring,1718,310,71,84
3,1305121398-0003-R.jpg,ring,1231,850,89,87
4,1305121398-0004-R.jpg,ring,815,1080,67,73


In [59]:
dataset = pd.concat([df, df_falc]).reset_index(drop=True)
dataset.head()

,filename,parasite_type,xmin,xmax,ymin,ymax
0,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1440,1540,1057,1158
1,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,1303,1403,868,971
2,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,900,1008,578,689
3,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,611,713,304,408
4,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,red blood cell,881,1003,198,312


In [61]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=dataset)

https://docs.google.com/spreadsheets/d/1xx1tepfO9ByOz6yOudXBIZpVonQhps0zmRtDsQPEyIY#gid=0


In [62]:
# prompt: créer un nouveau dossier et copie les image des dossier suivant dans le dossier créé. Les mettre aussi au même format .jpg : /content/malaria/images et /content/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis-master/Falciparum/img

import os
import shutil
from PIL import Image

def create_and_copy_images(source_dirs, destination_dir):
  """
  Creates a new directory and copies images from specified source directories to it, converting them to JPG format.

  Args:
      source_dirs: A list of directories containing the source images.
      destination_dir: The path to the directory where the images will be copied.
  """

  if not os.path.exists(destination_dir):
      os.makedirs(destination_dir)

  for source_dir in source_dirs:
    for filename in os.listdir(source_dir):
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Add more image extensions if needed
        source_path = os.path.join(source_dir, filename)
        destination_filename = os.path.splitext(filename)[0] + '.jpg'
        destination_path = os.path.join(destination_dir, destination_filename)
        try:
          img = Image.open(source_path)
          #img = img.convert('RGB')  # Ensure the image is in RGB format
          img.save(destination_path, 'JPEG')
        except Exception as e:
          print(f"Error processing image {source_path}: {e}")

# Define source directories and destination directory
source_dirs = ['/content/malaria/images', '/content/MP-IDB-The-Malaria-Parasite-Image-Database-for-Image-Processing-and-Analysis-master/Falciparum/img']
destination_dir = '/content/merged_images'

# Call the function to create the new directory and copy the images
create_and_copy_images(source_dirs, destination_dir)

print(f"Images copied to {destination_dir}")

Images copied to /content/merged_images


In [63]:
# prompt: zipper le dossier et le copier dans drive

import zipfile
from google.colab import drive

drive.mount('/content/drive')

def zip_and_copy_to_drive(folder_path, zip_filename, drive_folder_path):
  """
  Zips a folder and copies it to a specified location in Google Drive.

  Args:
      folder_path: The path to the folder to be zipped.
      zip_filename: The name of the zip file to be created.
      drive_folder_path: The path to the folder in Google Drive where the zip file will be copied.
  """

  shutil.make_archive(zip_filename, 'zip', folder_path)
  shutil.copy(zip_filename + '.zip', drive_folder_path)
  print(f"Folder '{folder_path}' zipped as '{zip_filename}.zip' and copied to '{drive_folder_path}'")

# Example usage:
folder_to_zip = '/content/merged_images'
zip_file_name = 'dataset_plasmodium_falc_vivax'
drive_destination_path = '/content/drive/My Drive/dataset_plasmodium_falc_vivax'

zip_and_copy_to_drive(folder_to_zip, zip_file_name, drive_destination_path)

Mounted at /content/drive
Folder '/content/merged_images' zipped as 'dataset_plasmodium_falc_vivax.zip' and copied to '/content/drive/My Drive/dataset_plasmodium_falc_vivax'
